

# 동양여우 ver 0.0.1
![alt text](resource/fox.png "Title")

### 이제 좀더 복잡한 테스팅을 해볼까요?
* 이제 복합적으로 선택해 나가는 것을 테스트 해볼거에요. (e.g., 대형주 중에서 영업이익이 높은 것 중에서 부채비율이 낮은 종목들)

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))


from core.constants import TableKey, TableValue
from core.constants import get_all_sub_tables


from core.strategy import Selector, SelectorType, SelectorSequence, Filter

selector_sequences = SelectorSequence(
    selectors=[Selector(type=SelectorType.HORIZON_RANGE, key=TableKey.MARKET_CAPITALIZATION,
                        value=(0.2, 0.5),
                        horizon=1,
                        is_absolute=False),
               # Selector(type=SelectorType.PROFILE, key=TableKey.Profile.SECTOR,
               #          value=[TableValue.Sector.TECHNOLOGY]),
               Selector(type=SelectorType.HORIZON_RANGE, key=TableKey.FinancialRatios.Quarter.DIVIDEND_YIELD,
                        value=(0.5, 1.0),
                        horizon=1,
                        is_absolute=False),
               ])

my_filter = Filter(selector_sequence_union=[selector_sequences], name="my filter")

baseline_filter = Filter(name='all stocks', selector_sequence_union = [SelectorSequence([Selector(type=SelectorType.PROFILE, key=TableKey.Profile.SECTOR, value=get_all_sub_tables(TableValue.Sector))])])

filter_list = [my_filter, baseline_filter]

### 투자 시뮬레이션 준비
* 자! 그럼 이제 **주어진 기간동안** (e.g., 1960년 ~ 2022년) 까지 일정한 주기로 (e.g., 3개월마다) 주식들을 골라서 갖고 있어보아요
* 고를때 A 라는 사람은 시가총액 상위 20% 만 B 라는 전략가는 상위 20%-40% 만 .. E 라는 사람은 하위 20 % 만 꾸준히 산다고 가정해볼게요.
* 일단 전체 모집단은 [NYSE, NASDAQ, ...? ]의 주식으로 해볼게요! (시간 관계상, 거래소 주식의 일부분만 대상으로 할게요)

In [ ]:
from datetime import date
from core.db_interface import DataBaseInterface
from core.strategy import Strategist
import pandas as pd
from tqdm import tqdm

db_interface = DataBaseInterface()

exchange_pool = ['NASDAQ']
horizon = (date(2010, 1, 1), date(2022, 8, 22))
freq = '6M'

initial_symbols = db_interface.get_stock_on_exchange(exchange_pool)
rebalance_dates = pd.date_range(horizon[0], horizon[1], freq=freq, inclusive='both')
strategist_list = []

tqdm_object = tqdm(total=len(filter_list) * len(rebalance_dates))
tqdm_object.set_description(f"동양여우가 {len(initial_symbols)}개의 주식 중에서 고르는 중 ")

with tqdm_object as pbar:
    for filter_ in filter_list:
        strategist_name = filter_.name
        strategist = Strategist(fetcher=db_interface, name=strategist_name, asset_pool=initial_symbols,
                                start_date=horizon[0], end_date=horizon[1])
        for rebalance_date in rebalance_dates:
            strategist.apply_filter(filter_in=filter_, date_in=rebalance_date.date())
            pbar.update(1)
        strategist_list.append(strategist)

print("다 되었어요!")

## 자, 그럼 우리가 여태 고른 주식들은 어떤 종목들이었는지 살펴볼까요? 😍
### 우선 한번 크게 봐볼까요?
* 각 전략별로 통 기간동안 고른 주식들의 섹터별 통계를 한번 봐볼게요.
* 맨 마지막 파이는 전제 대상 주식의 분포에요.

In [ ]:
from core.evaluate import show_selection, prepare_canvas
import matplotlib.pyplot as plt
import math

prepare_canvas(fontsize=10)
for idx, strategist in enumerate(strategist_list):
    plt.subplot(math.ceil(len(strategist_list) / 2), 2, idx + 1)
    show_selection(strategist, db_interface, radius=1.5, fontsize=15, labeldistacne=1.2, titlepad=-180)

# 자, 그럼이제 우리 각 전략들을 고수했을때 우리 잔고는 어떻게 되었는지 생각해볼까요?
* 처음 잔고를 1원이라고 가정해볼게요. 과연 이 기간동안 최종 잔고는 어떻게 되었을까요? 🤔
* 첫번째 그림이 각 전략을 계속 고수했을 때 잔고 상태입니다.

In [ ]:
from core.evaluate import get_all_symbols_price_history

df_all_price_history = get_all_symbols_price_history(strategist_list[-1], db_interface)
df_all_price_history = df_all_price_history.resample('6D').mean()

In [ ]:
from core.evaluate import show_comparative_report
show_comparative_report(strategist_list, df_all_price_history)

### 다음은, 각 전략별로 기간동안 무엇을 했는지 봐볼게요.
* 우리는 기간마다 전부 팔고, 다시 기준에 부합하는 주식을 새로 고르기 때문에 (rebalancing) 전략은 같아도 선택되는 주식은 늘 변할 거에요 !
* 왜냐면, 시가 총액 / PBR / PER 등등 매일 혹은 매 분기마다 변하는 것이니까요.

In [ ]:
from core.evaluate import show_sector_distribution_history, analyze_leaders, prepare_canvas
import matplotlib.pyplot as plt
strategist = strategist_list[0]
prepare_canvas(fontsize=11)
plt.subplot(2, 1, 1)
plt.title('섹터 비율 히스토리')
show_sector_distribution_history(strategist, db_interface)
plt.subplot(2, 1, 2)
plt.title('보유 종목 중 수익률 TOP 10')
df_leader_symbol, df_leader_return_ration =  analyze_leaders(strategist, df_all_price_history, True, {
            'fontsize': 10,
            'fontweight': 'bold',
            'fontfamily': 'sans'
        })